### Per Locus Mismatches in Comparison between Stack depth 3 v 10 simulation

** Original Notebook Text:**
<br>
In my combined data set for batch 1, I encountered a lot of missing genotypes in the Alaskan samples. The hypothesis is that this is because lower stack depth is preventing pstacks from genotyping individuals at a locus (not that the cat locus isn't being aligned to in the original .sam files). So I want to see if using a lower stack depth for pstacks is going to help genotype Alaskan individuals. But first I need to make sure that we can trust the genotypes generated with a lower stack depth.

I want to look at the per locus mismatches between the full size file / depth 10 and the half size file / depth 3. 
Even though I won't be using Marine's script for this project, I do want to compare both with and without the correction, just out of curiosity. 
I already have genotype files with only matched loci in them, so I can go straight to the comparison code. 


<br>
** Revisions:**
<br>
the code in the previous notebook did not correctly calculate genotyping error by locus for the recalled genotypes. This notebook does so, and also calculates genotype error per individual (rather than just per locus).
<br>

<br>
#### 4/23/2018


In [1]:
pwd

u'/mnt/hgfs/PCod-Compare-repo/notebooks/batch3_1'

In [2]:
cd ../

/mnt/hgfs/PCod-Compare-repo/notebooks


In [5]:
cd ../

/mnt/hgfs/PCod-Compare-repo


#### Original Code

In [ ]:
!python scripts/PostStacksFiltering/genepop_conversion_corrected.py \
stacks_b2_wgenome/batch_2_matched.CorrectedGenotypes_biallelic.txt \
stacks_b2_wgenome/batch_2_matched.CorrectedGenotypes_biallelic_gen.txt

This gave me two matrix files:
<br>
`batch_2_matched.CorrectedGenotypes_biallelic_gen.txt`
<br>
`batch_1_matched.CorrectedGenotypes_biallelic.txt`

<br>
First, I am going to create a dictionary using first loci, and then individuals, as keys and genotypes as values

In [14]:
####### create dictionary with loci ##############
batch1 = open("stacks_b1_wgenome/batch_1_matched.CorrectedGenotypes_biallelic.txt", "r")
batch2 = open("stacks_b2_wgenome/batch_2_matched.CorrectedGenotypes_biallelic_gen.txt", "r")

## initiate ordered dictionary so loci can be called with indices
import collections
loci_dict_b1 = collections.OrderedDict()
loci_dict_b2 = collections.OrderedDict()

## read loci in as keys
loci_list1 = batch1.readline().strip().split()
print len(loci_list1)
loci_list2 = batch2.readline().strip().split()
print len(loci_list2)

6937
6937


In [15]:
####### fill in dictionary with loci ##########
n_samples = 0
for line in batch1:
    genotypes = line.strip().split()[1:]
    if n_samples == 0:
        for i in range(0,len(genotypes)):
            loci_dict_b1[loci_list1[i]] = [genotypes[i]]
        n_samples += 1
    elif n_samples > 0:
        for i in range(0,len(genotypes)):
            geno_list = loci_dict_b1[loci_list1[i]]
            geno_list.append(genotypes[i])
            loci_dict_b1[loci_list1[i]] = geno_list
        n_samples += 1
print n_samples
batch1.close()

n_samples = 0
for line in batch2:
    genotypes = line.strip().split()[1:]
    if n_samples == 0:
        for i in range(0,len(genotypes)):
            loci_dict_b2[loci_list2[i]] = [genotypes[i]]
        n_samples += 1
    elif n_samples > 0:
        for i in range(0,len(genotypes)):
            geno_list = loci_dict_b2[loci_list2[i]]
            geno_list.append(genotypes[i])
            loci_dict_b2[loci_list2[i]] = geno_list
        n_samples += 1
print n_samples
batch2.close()

50
50


In [17]:
print len(loci_dict_b1["10004"])
print len(loci_dict_b2["10004"])

50
50


In [25]:
b1_genotypes = loci_dict_b1["10004"]
if b1_genotypes[1][0:2] == b1_genotypes[1][2:]:
    print "homozygote"

homozygote


In [28]:
########## compare genotypes at each locus #############
geno_codes = {}
for locus in loci_list1:
    b1_genotypes = loci_dict_b1[locus]
    b2_genotypes = loci_dict_b2[locus]
    coded = []
    ## add appropriate code to list based on comparison of two genotypes
    #--- 0 = both missing
    #--- 1 = batch 1 missing
    #--- 2 = batch 2 missing
    #--- 3 = both genotyped
    #--- 4 = both genotyped, matched
    #--- 5 = both genotyped, mismatch because of hom --> hom
    #--- 6 = both genotyped, mismatched because of hom --> het
    #--- 7 = both genotyped, mismatched because of het --> hom
    for i in range(0,len(b1_genotypes)):
        if b1_genotypes[i] == b2_genotypes[i]:
            # if genotypes matched and neither are missing
            if b1_genotypes[i] != "0000":
                coded.append(4)
            elif b1_genotypes[i] == "0000":
                coded.append(0)
        # if genotypes don't match
        elif b1_genotypes[i] != b2_genotypes[i]:
            # if they are mismatched because one is missing
            if b1_genotypes[i] == "0000":
                coded.append(1)
            elif b2_genotypes[i] == "0000":
                coded.append(2)
            # if they are mismatched and neither are missing
            else:
                # b1 het (b2 must be hom)
                if b1_genotypes[i][0:2] != b1_genotypes[i][2:]:
                    coded.append(7)
                # b1 hom
                elif b1_genotypes[i][0:2] == b1_genotypes[i][2:]:
                    # b2 hom
                    if b2_genotypes[i][0:2] == b2_genotypes[i][2:]:
                        coded.append(5)
                    # b2 het
                    elif b2_genotypes[i][0:2] != b2_genotypes[i][2:]:
                        coded.append(6)
    geno_codes[locus] = coded
    if len(coded) < 50:
        print "processed only ", len(coded), " samples at locus."

In [32]:
######## calculate mismatches ######
outfile = open("KOR_genotypes_depth3v10_MB_perlocus_mismatches_rerun.txt", "w")
outfile.write("locus\tn.pairs\tn.both.miss\tn.b1.miss\tn.b2.miss\tn.both.genod\tn.matched\tn.mismatch.hom2hom\tn.mismatch.het2hom\tn.mismatch.hom2het\n")


for locus in loci_list1:
    coded = geno_codes[locus]
    n_pairs = len(coded)
    n_miss = len([i for i in coded if i == 0])
    n_missb1 = len([i for i in coded if i == 1])
    n_missb2 = len([i for i in coded if i == 2])
    n_matched = len([i for i in coded if i == 4])
    n_mismatch_homhom = len([i for i in coded if i == 5])
    n_mismatch_homhet = len([i for i in coded if i == 6])
    n_mismatch_hethom = len([i for i in coded if i == 7])
    n_genod = n_matched + n_mismatch_homhom + n_mismatch_homhet + n_mismatch_hethom
    outfile.write(locus + "\t" + str(n_pairs) + "\t" + str(n_miss) + "\t" + str(n_missb1) + "\t" + str(n_missb2) + "\t" + str(n_genod) + "\t")
    outfile.write(str(n_matched) + "\t" + str(n_mismatch_homhom) + "\t" + str(n_mismatch_homhet) + "\t" + str(n_mismatch_hethom) + "\n")
outfile.close()

In [33]:
!head KOR_genotypes_depth3v10_MB_perlocus_mismatches_rerun.txt

locus	n.pairs	n.both.miss	n.b1.miss	n.b2.miss	n.both.genod	n.matched	n.mismatch.hom2hom	n.mismatch.het2hom	n.mismatch.hom2het
10000	50	5	0	3	42	42	0	0	0
10004	50	1	0	2	47	46	0	0	1
10011	50	5	0	6	39	39	0	0	0
10012	50	4	46	0	0	0	0	0	0
10016	50	3	0	1	46	46	0	0	0
10018	50	1	0	2	47	0	43	4	0
10019	50	1	0	3	46	46	0	0	0
10023	50	0	0	1	49	49	0	0	0
10026	50	7	0	1	42	39	0	1	2


### Mismatches per Individual

#### Recalled Genotypes

In [40]:
####### create dictionary with individs ##############
batch1 = open("stacks_b1_wgenome/batch_1_matched.CorrectedGenotypes_biallelic.txt", "r")
batch2 = open("stacks_b2_wgenome/batch_2_matched.CorrectedGenotypes_biallelic_gen.txt", "r")

## initiate ordered dictionary so loci can be called with indices
import collections
samples_dict_b1 = collections.OrderedDict()
samples_dict_b2 = collections.OrderedDict()

## read loci in as keys
batch1.readline()
batch2.readline()
for line in batch1:
    linelist = line.strip().split()
    samples_dict_b1[linelist[0]] = linelist[1:]
batch1.close()

for line in batch2:
    linelist = line.strip().split()
    sampID = linelist[0].strip("_subset")
    samples_dict_b2[sampID] = linelist[1:]
batch2.close()

len([i for i in samples_dict_b1.keys() if i in samples_dict_b2.keys()])

50

In [45]:
######## compare genotypes in each individual #########
samples_geno_codes = {}

for sample in samples_dict_b1.keys():
    b1_genotypes = samples_dict_b1[sample]
    b2_genotypes = samples_dict_b2[sample]
    coded = []
    ## add appropriate code to list based on comparison of two genotypes
    #--- 0 = both missing
    #--- 1 = batch 1 missing
    #--- 2 = batch 2 missing
    #--- 3 = both genotyped
    #--- 4 = both genotyped, matched
    #--- 5 = both genotyped, mismatch because of hom --> hom
    #--- 6 = both genotyped, mismatched because of hom --> het
    #--- 7 = both genotyped, mismatched because of het --> hom
    for i in range(0,len(b1_genotypes)):
        if b1_genotypes[i] == b2_genotypes[i]:
            # if genotypes matched and neither are missing
            if b1_genotypes[i] != "0000":
                coded.append(4)
            elif b1_genotypes[i] == "0000":
                coded.append(0)
        # if genotypes don't match
        elif b1_genotypes[i] != b2_genotypes[i]:
            # if they are mismatched because one is missing
            if b1_genotypes[i] == "0000":
                coded.append(1)
            elif b2_genotypes[i] == "0000":
                coded.append(2)
            # if they are mismatched and neither are missing
            else:
                # b1 het (b2 must be hom)
                if b1_genotypes[i][0:2] != b1_genotypes[i][2:]:
                    coded.append(7)
                # b1 hom
                elif b1_genotypes[i][0:2] == b1_genotypes[i][2:]:
                    # b2 hom
                    if b2_genotypes[i][0:2] == b2_genotypes[i][2:]:
                        coded.append(5)
                    # b2 het
                    elif b2_genotypes[i][0:2] != b2_genotypes[i][2:]:
                        coded.append(6)
    samples_geno_codes[sample] = coded
    if len(coded) < 6937:
        print "processed only ", len(coded), " samples at locus."

In [51]:
######## calculate mismatches ######
outfile = open("KOR_genotypes_depth3v10_MB_byIndivid_mismatches.txt", "w")
outfile.write("sample\tn.loci\tn.both.miss\tn.b1.miss\tn.b2.miss\tn.both.genod\tn.matched\tn.mismatch.hom2hom\tn.mismatch.het2hom\tn.mismatch.hom2het\n")


for sample in samples_geno_codes.keys():
    coded = samples_geno_codes[sample]
    n_loci = len(coded)
    n_miss = len([i for i in coded if i == 0])
    n_missb1 = len([i for i in coded if i == 1])
    n_missb2 = len([i for i in coded if i == 2])
    n_matched = len([i for i in coded if i == 4])
    n_mismatch_homhom = len([i for i in coded if i == 5])
    n_mismatch_homhet = len([i for i in coded if i == 6])
    n_mismatch_hethom = len([i for i in coded if i == 7])
    n_genod = n_matched + n_mismatch_homhom + n_mismatch_homhet + n_mismatch_hethom
    outfile.write(sample + "\t" + str(n_loci) + "\t" + str(n_miss) + "\t" + str(n_missb1) + "\t" + str(n_missb2) + "\t" + str(n_genod) + "\t")
    outfile.write(str(n_matched) + "\t" + str(n_mismatch_homhom) + "\t" + str(n_mismatch_homhet) + "\t" + str(n_mismatch_hethom) + "\n")
outfile.close()


<br>
#### Stacks genotypes


In [52]:
####### create dictionary with loci ##############
batch1 = open("stacks_b1_wgenome/batch_1_matched_matrix.txt", "r")
batch2 = open("stacks_b2_wgenome/batch_2_matched_matrix.txt", "r")

## initiate ordered dictionary so loci can be called with indices
import collections
samples_dict_b1 = collections.OrderedDict()
samples_dict_b2 = collections.OrderedDict()

## read loci in as keys
batch1.readline()
batch2.readline()
for line in batch1:
    linelist = line.strip().split()
    samples_dict_b1[linelist[0]] = linelist[1:]
batch1.close()

for line in batch2:
    linelist = line.strip().split()
    sampID = linelist[0].strip("_subset")
    samples_dict_b2[sampID] = linelist[1:]
batch2.close()

len([i for i in samples_dict_b1.keys() if i in samples_dict_b2.keys()])

50

In [53]:
######## compare genotypes in each individual #########
samples_geno_codes = {}

for sample in samples_dict_b1.keys():
    b1_genotypes = samples_dict_b1[sample]
    b2_genotypes = samples_dict_b2[sample]
    coded = []
    ## add appropriate code to list based on comparison of two genotypes
    #--- 0 = both missing
    #--- 1 = batch 1 missing
    #--- 2 = batch 2 missing
    #--- 3 = both genotyped
    #--- 4 = both genotyped, matched
    #--- 5 = both genotyped, mismatch because of hom --> hom
    #--- 6 = both genotyped, mismatched because of hom --> het
    #--- 7 = both genotyped, mismatched because of het --> hom
    for i in range(0,len(b1_genotypes)):
        if b1_genotypes[i] == b2_genotypes[i]:
            # if genotypes matched and neither are missing
            if b1_genotypes[i] != "0000":
                coded.append(4)
            elif b1_genotypes[i] == "0000":
                coded.append(0)
        # if genotypes don't match
        elif b1_genotypes[i] != b2_genotypes[i]:
            # if they are mismatched because one is missing
            if b1_genotypes[i] == "0000":
                coded.append(1)
            elif b2_genotypes[i] == "0000":
                coded.append(2)
            # if they are mismatched and neither are missing
            else:
                # b1 het (b2 must be hom)
                if b1_genotypes[i][0:2] != b1_genotypes[i][2:]:
                    coded.append(7)
                # b1 hom
                elif b1_genotypes[i][0:2] == b1_genotypes[i][2:]:
                    # b2 hom
                    if b2_genotypes[i][0:2] == b2_genotypes[i][2:]:
                        coded.append(5)
                    # b2 het
                    elif b2_genotypes[i][0:2] != b2_genotypes[i][2:]:
                        coded.append(6)
    samples_geno_codes[sample] = coded
    if len(coded) < 6937:
        print "processed only ", len(coded), " loci at sample."

In [54]:
######## calculate mismatches ######
outfile = open("KOR_genotypes_depth3v10_byIndivid_mismatches.txt", "w")
outfile.write("sample\tn.loci\tn.both.miss\tn.b1.miss\tn.b2.miss\tn.both.genod\tn.matched\tn.mismatch.hom2hom\tn.mismatch.het2hom\tn.mismatch.hom2het\n")


for sample in samples_geno_codes.keys():
    coded = samples_geno_codes[sample]
    n_loci = len(coded)
    n_miss = len([i for i in coded if i == 0])
    n_missb1 = len([i for i in coded if i == 1])
    n_missb2 = len([i for i in coded if i == 2])
    n_matched = len([i for i in coded if i == 4])
    n_mismatch_homhom = len([i for i in coded if i == 5])
    n_mismatch_homhet = len([i for i in coded if i == 6])
    n_mismatch_hethom = len([i for i in coded if i == 7])
    n_genod = n_matched + n_mismatch_homhom + n_mismatch_homhet + n_mismatch_hethom
    outfile.write(sample + "\t" + str(n_loci) + "\t" + str(n_miss) + "\t" + str(n_missb1) + "\t" + str(n_missb2) + "\t" + str(n_genod) + "\t")
    outfile.write(str(n_matched) + "\t" + str(n_mismatch_homhom) + "\t" + str(n_mismatch_homhet) + "\t" + str(n_mismatch_hethom) + "\n")
outfile.close()